<a href="https://colab.research.google.com/github/raiaiaia/llm-code-review-clj/blob/main/semantic_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FILE_PATH = '/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/rag_model_output_final.csv'
df = pd.read_csv(FILE_PATH)

In [ ]:
print(df.columns)

Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'model_comment'],
      dtype='object')


In [ ]:
%%capture
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

In [ ]:
def get_embedding_function(text: str) -> list:
  inputs = tokenizer(
      text,
      return_tensors="pt",
      truncation=True,
      padding=True
      )

  with torch.no_grad():
    outputs = model(**inputs)
  embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

  return embeddings

In [ ]:
def calc_cos_sim(row, model):
  model_emb = get_embedding_function(row['model_comment'])
  human_emb = get_embedding_function(row['content'])

  return cosine_similarity(model_emb.reshape(1, -1), human_emb.reshape(1, -1))[0][0]

In [ ]:
tqdm.pandas()
df['cos_sim'] = df.progress_apply(calc_cos_sim, axis=1, args=(model,))
similarity_df = df.sort_values(by='cos_sim', ascending=False)
similarity_df.to_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/similarity_rag_model.csv', index=False)

100%|██████████| 5105/5105 [22:42<00:00,  3.75it/s]
